In [11]:
import pandas as pd
import datetime
import numpy as np
import argparse
import math
import multiprocessing as mp
import time

In [56]:
parser = argparse.ArgumentParser(description='Calculating hysplit trajs.')

parser.add_argument("--climate", choices=["hist", "pgw"], required=True, type=str, help="This is the hist or pgw choice.")
parser.add_argument("--month", required=True, type=int, help="Storm month for trajectory calculation.")
parser.add_argument("--working", required=True, type=str, help="The hysplit working folder.")
parser.add_argument("--ens", required=True, type=int, help="Ensemble number.")

args=parser.parse_args()

usage: ipykernel_launcher.py [-h] --climate {hist,pgw} --month MONTH --working
                             WORKING --ens ENS
ipykernel_launcher.py: error: the following arguments are required: --climate, --month, --working, --ens


SystemExit: 2

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
which_climate=args.climate
which_month=args.month
which_working=args.working
ens_number=args.ens

In [49]:
csv_file=pd.read_csv(f'/glade/work/bpoujol/Moisture_tracking/trajectory_information_pgw.csv')

In [50]:
dates=[]
for datetime_string in csv_file['TIME (UTC)'].values:
    dates.append(datetime.datetime.strptime(datetime_string, '%Y-%m-%d_%H:%M:%S'))
ready_dates=pd.to_datetime(np.array(dates))

csv_file['YEAR']=ready_dates.year
csv_file['MONTH']=ready_dates.month
csv_file['DAY']=ready_dates.day
csv_file['HOUR']=ready_dates.hour

csv_file=csv_file[csv_file['MONTH']==6]

In [52]:
csv_file['TRAJ. ID'].values

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         23,   24,   25,   26,   27,   28,   29,   30,   31,   32,  121,
        122,  123,  124,  125,  126,  127,  128,  129,  130,  131,  132,
        133,  134,  135,  136,  137,  138,  139,  140,  141,  142,  143,
        144,  145,  146,  147,  148,  149,  150,  151,  152,  153,  154,
        295,  296,  297,  298,  299,  300,  301,  302,  303,  304,  305,
        306,  307,  308,  309,  310,  311,  312,  313,  314,  315,  316,
        317,  318,  319,  320,  321,  322,  323,  324,  325,  433,  434,
        435,  436,  437,  438,  439,  440,  441,  442,  443,  444,  445,
        446,  447,  448,  449,  450,  451,  452,  453,  454,  455,  456,
        457,  534,  535,  536,  537,  538,  539,  540,  541,  542,  543,
        544,  545,  546,  547,  548,  549,  550,  551,  552,  553,  554,
        555,  556,  557,  558,  559,  560,  561,  5

In [46]:
def ens_create(ens_num, lat, lon):
    if ens_num==1:
        return compute_displacement(lat, lon, dist=1, bear=90)
    if ens_num==2:
        return compute_displacement(lat, lon, dist=2, bear=90)
    if ens_num==3:
        return compute_displacement(lat, lon, dist=3, bear=90)
    if ens_num==4:
        return compute_displacement(lat, lon, dist=1, bear=270)
    if ens_num==5:
        return compute_displacement(lat, lon, dist=2, bear=270)
    if ens_num==6:
        return compute_displacement(lat, lon, dist=3, bear=270)
    if ens_num==7:
        return compute_displacement(lat, lon, dist=1, bear=180)
    if ens_num==8:
        return compute_displacement(lat, lon, dist=2, bear=180)
    if ens_num==9:
        return compute_displacement(lat, lon, dist=3, bear=180)
    if ens_num==10:
        return compute_displacement(lat, lon, dist=1, bear=0)
    if ens_num==11:
        return compute_displacement(lat, lon, dist=2, bear=0)
    if ens_num==12:
        return compute_displacement(lat, lon, dist=3, bear=0)
    if ens_num==13:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==14:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==15:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==16:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==17:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==18:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==19:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==20:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==21:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==22:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==23:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==24:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==25:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==26:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==27:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==28:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==29:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==30:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=0)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==31:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==32:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==33:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==34:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==35:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==36:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==37:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=90)
    if ens_num==38:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=90)
    if ens_num==39:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=90)
    if ens_num==40:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==41:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==42:
        newlat, newlon=compute_displacement(lat, lon, dist=1, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==43:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==44:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==45:
        newlat, newlon=compute_displacement(lat, lon, dist=2, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=270)
    if ens_num==46:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=1, bear=270)
    if ens_num==47:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=2, bear=270)
    if ens_num==48:
        newlat, newlon=compute_displacement(lat, lon, dist=3, bear=180)
        return compute_displacement(newlat, newlon, dist=3, bear=270)

In [47]:
def compute_displacement(lat, lon, dist, bear):
    R = 6378.1 #Radius of the Earth (km)
    brng = math.radians(bear) #Bearing is 90 degrees converted to radians.
    d = dist #Distance in km
    lat1 = math.radians(lat) #Current lat point converted to radians
    lon1 = math.radians(lon) #Current long point converted to radians
    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
           math.cos(lat1)*math.sin(d/R)*math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
           math.cos(d/R)-math.sin(lat1)*math.sin(lat2))
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    return lat2, lon2

In [ ]:
def call_hysplit_func(basename, working_dir, output_dir, meteo_dir, years, months, hours, altitudes, location, runtime, day1, day2, hysplit_dir):
    
    import time
    from pysplit.trajectory_generator import generate_bulktraj
    generate_bulktraj(basename=basename, 
                      hysplit_working=working_dir, 
                      output_dir=output_dir, 
                      meteo_dir=meteo_dir, 
                      years=years, 
                      months=months, 
                      hours=hours, 
                      altitudes=altitudes, 
                      coordinates=location, 
                      run=runtime,
                      meteoyr_2digits=False, outputyr_2digits=False,         
                      monthslice=slice(day1, day2, 1),
                      meteo_bookends=([1] , [1]),
                      get_reverse=False, get_clipped=False, hysplit=hysplit_dir)
    time.sleep(30)

In [54]:
############################################################################
############################################################################

#where is hysplit working folder?
working_dir = f'/glade/u/home/molina/hysplit/trunk/{which_working}' 
#where is arl format meteo data?
meteo_dir = f'/glade/scratch/molina/basile/{which_climate}'
#where is hysplit model executable?
hysplit_dir=r'/glade/u/home/molina/hysplit/trunk/exec/hyts_std'
#where to put trajs?
output_dir=f'/glade/scratch/molina/basile/{which_climate}_traj/'

############################################################################
############################################################################

runtime = -240

basename = []
years = []
months = []
hours = []
location = []
altitudes = []

pool1=mp.Pool(35)

for i in range(len(csv_file)):

    print(i)
    basename = 'trajid'+str(csv_file.iloc[i][0])+'_subregion'+str(csv_file.iloc[i][2])+'_'+'ens'+str(ens_number)+'_'

    years = [csv_file.iloc[i][8]]        
    months = [csv_file.iloc[i][9]]
    hours = [csv_file.iloc[i][11]]
    location = ens_create(ens_number, csv_file.iloc[i][4], csv_file.iloc[i][5])
    altitudes = [csv_file.iloc[i][6]] 
    day1 = (csv_file.iloc[i][10]-1)
    day2 = csv_file.iloc[i][10]

    pool1.apply_async(call_hysplit_func, 
                      args=([basename, working_dir, output_dir, meteo_dir, years, months, hours, altitudes, location, runtime, day1, day2, hysplit_dir]))
    time.sleep(5)

pool1.close()
pool1.join()
print(f"Completed the jobs.")

############################################################################
############################################################################
############################################################################ 

SyntaxError: positional argument follows keyword argument (<ipython-input-54-48ecd5d028bf>, line 38)

In [20]:
for i in range(len(csv_file)):
    dir="work"+str(i)
    print(dir)

work0
work1
work2
work3
work4
work5
work6
work7
work8
work9
work10
work11
work12
work13
work14
work15
work16
work17
work18
work19
work20
work21
work22
work23
work24
work25
work26
work27
work28
work29
work30
work31
work32
work33
work34
work35
work36
work37
work38
work39
work40
work41
work42
work43
work44
work45
work46
work47
work48
work49
work50
work51
work52
work53
work54
work55
work56
work57
work58
work59
work60
work61
work62
work63
work64
work65
work66
work67
work68
work69
work70
work71
work72
work73
work74
work75
work76
work77
work78
work79
work80
work81
work82
work83
work84
work85
work86
work87
work88
work89
work90
work91
work92
work93
work94
work95
work96
work97
work98
work99
work100
work101
work102
work103
work104
work105
work106
work107
work108
work109
work110
work111
work112
work113
work114
work115
work116
work117
work118
work119
work120
work121
work122
work123
work124
work125
work126
work127
work128
work129
work130
work131
work132
work133
work134
work135
work136
work137
work13